In [ ]:
# --- [CELDA 1: SETUP Y PREPARACIÓN PARA ENTRENAMIENTO] ---

# 1. Montar Google Drive para acceder al dataset
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
print("✅ Google Drive montado.")

# 2. Instalar la librería de Ultralytics (contiene YOLOv8)
# El flag '-q' (quiet) reduce el output de la instalación.
!pip install ultralytics -q
print("✅ Librería Ultralytics instalada.")

# 3. Importar librerías y verificar el entorno
import ultralytics
from ultralytics import YOLO
from pathlib import Path
import torch

# Esta función de Ultralytics revisa que la GPU esté disponible y todo esté correcto
# Deberías ver 'CUDA Device(s): 0...' y el nombre de tu GPU.
print("\n--- Verificación del Entorno ---")
ultralytics.checks()
print("------------------------------\n")


# 4. Definir la ruta al archivo de configuración del dataset
# Este es el archivo .yaml que creamos y guardamos en Google Drive.
DATASET_YAML_PATH = Path('/content/drive/MyDrive/Coffee_Unified_Seg_v1/data.yaml')

# Verificar que el archivo de configuración del dataset existe
if DATASET_YAML_PATH.exists():
    print(f"✅ Archivo de configuración del dataset encontrado en: {DATASET_YAML_PATH}")
else:
    # Si ves este error, revisa que el nombre de la carpeta en Drive sea exactamente 'Coffee_Unified_Seg_v1'
    print(f"❌ ¡ERROR! No se encontró el archivo 'data.yaml' en la ruta especificada.")
    print("   Asegúrate de que la ruta sea correcta y el dataset esté en tu Google Drive.")
    torch.hub.Stoppable() # Detiene la ejecución si no se encuentra el dataset.

# 5. Cargar un modelo pre-entrenado de segmentación
# Usamos 'yolov8n-seg.pt' (versión 'nano', la más rápida).
# Para mayor precisión puedes usar 'yolov8s-seg.pt' (small) o 'yolov8m-seg.pt' (medium).
model = YOLO('yolov8n-seg.pt')
print(f"\n✅ Modelo {model.ckpt_path.split('/')[-1]} cargado y listo para entrenar.")
print("\n===================================================================")
print("  TODO LISTO. Descomenta y ejecuta la línea 'model.train' para empezar.")
print("===================================================================\n")


Ultralytics 8.3.157 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 41.6/235.7 GB disk)
------------------------------

✅ Archivo de configuración del dataset encontrado en: /content/drive/MyDrive/Coffee_Unified_Seg_v1/data.yaml


100%|██████████| 6.74M/6.74M [00:00<00:00, 75.4MB/s]


✅ Modelo yolov8n-seg.pt cargado y listo para entrenar.

  TODO LISTO. Descomenta y ejecuta la línea 'model.train' para empezar.



In [ ]:
# --- FASE 1: ENTRENAMIENTO DE MÁXIMA CAPACIDAD (640px) ---

import os
import shutil
import torch
import traceback
from pathlib import Path
from ultralytics import YOLO

print("--- [FASE 1] CONFIGURANDO ENTRENAMIENTO DE ALTA CAPACIDAD ---")

# 1. Montaje de Drive y Verificación de GPU
# =================================================
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
if not torch.cuda.is_available(): raise RuntimeError("❌ GPU NO DETECTADA")
print("\n✅ GPU detectada. ¡Listos para la Fase 1!")

# 2. PARÁMETROS Y RUTAS PARA FASE 1
# =================================================
print("\n--- Definiendo parámetros y rutas para Fase 1 ---")

DATASET_YAML_PATH = "/content/drive/MyDrive/Coffee_Unified_Seg_v1/data.yaml"
MODELS_DESTINATION_DIR = Path('/content/drive/MyDrive/YOLOv8_Trained_Models/')

# [CAMBIO] Aumentamos la capacidad del modelo a 'x' (extra-large).
MODEL_VARIANT = 'yolov8x-seg.pt'
PROJECT_NAME = 'Coffee_Seg_High_Performance'
IMG_SIZE = 640
BATCH_SIZE = -1 # Mantenemos Auto-Batch para la A100

# [CAMBIO] Más épocas y paciencia para una convergencia completa.
EPOCHS = 250
PATIENCE = 50
OPTIMIZER = 'AdamW'

# [CAMBIO] Ajustamos la tasa de aprendizaje y aumentos.
LEARNING_RATE = 0.007 # Un poco más bajo para mayor estabilidad con el modelo 'x'.
COPY_PASTE_PROB = 0.6

RUN_NAME = f'F1_{MODEL_VARIANT.split("-")[0]}_img{IMG_SIZE}_b{str(BATCH_SIZE)}'

print(f"\nConfiguración Fase 1: \n  - Modelo: {MODEL_VARIANT}\n  - Run: {RUN_NAME}")
MODELS_DESTINATION_DIR.mkdir(exist_ok=True)


# 3. BUCLE DE ENTRENAMIENTO
# =================================================
model = YOLO(MODEL_VARIANT)

try:
    print(f"\n>>> Iniciando entrenamiento Fase 1: {RUN_NAME} <<<")
    model.train(
        data=DATASET_YAML_PATH,
        project=PROJECT_NAME,
        name=RUN_NAME,
        epochs=EPOCHS,
        patience=PATIENCE,
        imgsz=IMG_SIZE,
        batch=BATCH_SIZE,
        optimizer=OPTIMIZER,
        lr0=LEARNING_RATE,
        copy_paste=COPY_PASTE_PROB,
        mosaic=1.0,
        mixup=0.1,
        close_mosaic=20,
        exist_ok=True,
        seed=42
    )
    print(f"\n--- ✅ FASE 1 COMPLETADA NORMALMENTE ---")

except (Exception, KeyboardInterrupt) as e:
    print(f"\n--- 🛑 ENTRENAMIENTO FASE 1 INTERRUMPIDO O FALLIDO: {type(e).__name__} ---")
    if not isinstance(e, KeyboardInterrupt):
        traceback.print_exc()

finally:
    print("\n--- Guardando respaldo de Fase 1... ---")
    try:
        source_model_path = Path(f'./{PROJECT_NAME}/{RUN_NAME}/weights/best.pt')
        destination_model_path = MODELS_DESTINATION_DIR / f'best_{RUN_NAME}.pt'
        if source_model_path.exists():
            shutil.copy2(source_model_path, destination_model_path)
            print(f"✅ ¡Éxito! Mejor modelo de Fase 1 guardado en: {destination_model_path}")
        else:
            print("🟡 AVISO: No se encontró 'best.pt' en Fase 1.")
    except Exception as final_e:
        print(f"❌ Error durante guardado de Fase 1: {final_e}")

print("\n--- FIN DEL PROCESO FASE 1 ---")

--- [FASE 1] CONFIGURANDO ENTRENAMIENTO DE ALTA CAPACIDAD ---
Mounted at /content/drive

✅ GPU detectada. ¡Listos para la Fase 1!

--- Definiendo parámetros y rutas para Fase 1 ---

Configuración Fase 1: 
  - Modelo: yolov8x-seg.pt
  - Run: F1_yolov8x_img640_b-1


100%|██████████| 137M/137M [00:00<00:00, 211MB/s]



>>> Iniciando entrenamiento Fase 1: F1_yolov8x_img640_b-1 <<<
Ultralytics 8.3.157 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=20, cls=0.5, conf=None, copy_paste=0.6, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Coffee_Unified_Seg_v1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=250, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.007, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolov8x-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=F1_yolov8x_img640_b-1, nbs=64, nms=False, opse

train: Scanning /content/drive/MyDrive/Coffee_Unified_Seg_v1/train/labels.cache... 762 images, 0 backgrounds, 0 corrupt: 100%|██████████| 762/762 [00:00<?, ?it/s]

train: /content/drive/MyDrive/Coffee_Unified_Seg_v1/train/images/coffe_fobv2_v5_20240324_130202_jpg.rf.8e7b1f5a03700f46178efd43fc4b379f.jpg: 2 duplicate labels removed
train: /content/drive/MyDrive/Coffee_Unified_Seg_v1/train/images/coffe_fobv2_v5_20240324_131636_jpg.rf.571f2ceaab8c7a42c273bb393bd728e2.jpg: 1 duplicate labels removed
train: /content/drive/MyDrive/Coffee_Unified_Seg_v1/train/images/coffe_fobv2_v5_20240324_131636_jpg.rf.824635f78f37f4a1112a0224522af310.jpg: 1 duplicate labels removed
train: /content/drive/MyDrive/Coffee_Unified_Seg_v1/train/images/coffe_fobv2_v5_original_20240324_131636_jpg.rf.6ae45ed950a07fb80e96bee2e9a0e146.jpg: 1 duplicate labels removed
train: /content/drive/MyDrive/Coffee_Unified_Seg_v1/train/images/coffee-cherries-qpktr_v1_2_20211204_111100_jpg.rf.1297b8acd5bfc53055a31556c62de7fe.jpg: 1 duplicate labels removed
train: /content/drive/MyDrive/Coffee_Unified_Seg_v1/train/images/coffee-cherries-qpktr_v1_IMG_20230824_102954_jpg.rf.6fa38f627ca098c6b4ae4e

...

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):  22%|██▏       | 2/9 [00:00<00:01,  4.87it/s]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:02<00:00,  3.19it/s]


                   all         70       1414      0.821      0.819      0.875      0.835      0.825      0.822       0.88      0.799
              Inmaduro         60        644      0.925      0.932      0.957      0.909      0.925      0.932      0.958      0.855
                Pinton         23        214      0.808      0.897      0.947      0.914      0.808      0.897      0.947      0.877
                Maduro         26        306      0.848      0.778      0.877      0.825      0.848      0.778      0.877      0.766
           Sobremaduro         18         63      0.614      0.524      0.621      0.614      0.633       0.54      0.639        0.6
                  Seco         57        187      0.911      0.963      0.973      0.914      0.911      0.963      0.978      0.896
Speed: 0.2ms preprocess, 12.3ms inference, 0.1ms loss, 6.6ms postprocess per image
Results saved to Coffee_Seg_High_Performance/F1_yolov8x_img640_b-1

--- ✅ FASE 1 COMPLETADA NORMALMENTE ---

--- Guarda